In [2]:
# bring in our LLAMA_CLOUD_API_KEY
from dotenv import load_dotenv
load_dotenv()

import nest_asyncio

nest_asyncio.apply()

In [12]:
from llama_parse import LlamaParse 
import json

llamaparse = LlamaParse(premium_mode=True)
parsed_result = llamaparse.get_json_result("./data/redacao_1.jpg")

Started parsing the file under job_id df122324-8193-4a8a-b140-4c76d73cb4b0


In [13]:
from IPython.display import display, Markdown


display(Markdown(parsed_result[0]['pages'][0]['md']))

# FOLHA DE REDAÇÃO
EXAME NACIONAL DO ENSINO MÉDIO - ENEM 2023

**Número de Inscrição**: 231005093712  
**Sequencial**: 028  
**Sala**: 617  
**Data de Nascimento**: 29/10/1988

### INSTRUÇÕES
1. Transcreva a sua redação com caneta esferográfica de tinta preta, fabricada em material transparente.
2. Escreva a sua redação com letra legível. No caso de erro, risque, com um traço simples, a palavra, a frase, o trecho ou o sinal gráfico e escreva, em seguida, o respectivo substitutivo.
3. Não será avaliado texto escrito em local indevido. Respeite rigorosamente as margens.

Historicamente, o mundo do trabalho foi dominado pelos homens, restando para as mulheres, prin-
cipalmente, o trabalho doméstico e de cuidado com a família. Com efeito, no Brasil não foi diferente, e,
ainda hoje, existem desafios que precisam ser enfrentados para reduzir a invisibilidade do trabalho de
cuidado realizado pela mulher no país, uma vez que esse grupo da sociedade permanece em situação de
exclusão e resistência no mercado trabalhista. Diante desse cenário, é fundamental compreender as
causas desse revés, dentre as quais a desigualdade social e o patriarcalismo enraizado na sociedade
são fatores agravantes dessa problemática.

Em uma primeira análise, destaca-se a desigualdade social como um dos fatores a ser enfrentado pa-
ra amenizar a invisibilidade do trabalho de cuidado realizado pela mulher no Brasil. Nesse sentido,
a escritora Carolina Maria de Jesus, em sua obra literária "Quarto de Despejo", escrita em meados do sé-
culo XX, traz uma denúncia social, com uma visão feminina, de uma moradora de favela, na qual re-
trata a invisibilidade das pessoas que vivem em regiões desprivilegiadas pelo poder público e que precisam
se submeter a situações degradantes para conseguir sobreviver e não morrer de fome. Sob essa ótica, pa-
rece-se que a desigualdade social gerada pela ausência do amparo estatal para as camadas mais pobres da
população ocasiona na aceitação de trabalhos mal remunerados e informais, principalmente pelas mulhe-
res. Assim, é fundamental que haja políticas públicas que reconheçam a importância do trabalho de cuidado no
país e tragam garantias para essa atividade que será cada vez mais demandada com o envelhecimento da população.

Outrossim, salienta-se que o patriarcalismo enraizado na sociedade brasileira é outro fator que contribui pa-
ra a falta de visibilidade do trabalho de cuidado desempenhado pela mulher no país, sendo a desconstrução dessa
cultura de inferioridade feminina mais um desafio a ser superado para solucionar esse gravame social. Segundo
preceitua a Constituição Federal, em seu artigo 5º, é objetivo do Estado a construção de uma sociedade justa e igua-
litária, sem distinção de gênero, raça, etc. No entanto, as mulheres ainda são prejudicadas quanto aos direitos
trabalhistas, recebendo salários menores e não sendo valorizadas especialmente nos trabalhos realizados em am-
bientes domésticos. Dessa forma, é preciso que haja uma mudança cultural no pensamento patriarcal a fim de ampliar
a importância desse trabalho de cuidado feminino com o reconhecimento das garantias constitucionais.

É urgente, portanto, que medidas precisam ser tomadas para solucionar esse problema. Para tanto, o Congresso
Nacional, por meio da elaboração de leis específicas, deve criar normas que assegurem a visibilidade do trabalho de
cuidado existente no país, executado majoritariamente por mulheres, com a regulamentação de direitos trabalhis-
tas para essa categoria, a fim de que essas trabalhadoras possam ter uma vida digna, uma jornada legal de traba-
lho e uma remuneração justa que lhes permita ter acesso à moradia, alimentação, saúde e lazer.

In [16]:
all_content_md = parsed_result[0]['pages'][0]['md']

In [22]:
from pydantic import BaseModel, Field
from typing import List

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="gpt-4o-2024-08-06", # 100% json output
    temperature=0,
)

system_prompt = """
Você vai receber uma redação. Por favor, extraia o texto da redação, o nome do aluno, a data de nascimento e a sala
"""

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt), 
            ("human", "All document: \n\n {document}")
        ]
)

model_name = "gpt-4o-2024-08-06"

class GetSchema(BaseModel):
    """Extrai metadado de uma redação"""
    
    redacao_texto: str = Field(description="Texto da redação")
    nome: str = Field(description="Nome do aluno ou participante")
    data_nascimento: str = Field(description="Data de nascimento do aluno")
    sala: str = Field(description="Sala do aluno")

llm_with_tools_extraction = llm.bind_tools([GetSchema]) #, strict=True)
chain_structured_extraction = prompt | llm_with_tools_extraction

In [23]:
response = chain_structured_extraction.invoke({"document": all_content_md})

In [25]:
response.tool_calls[0]['args']

{'redacao_texto': 'Historicamente, o mundo do trabalho foi dominado pelos homens, restando para as mulheres, principalmente, o trabalho doméstico e de cuidado com a família. Com efeito, no Brasil não foi diferente, e, ainda hoje, existem desafios que precisam ser enfrentados para reduzir a invisibilidade do trabalho de cuidado realizado pela mulher no país, uma vez que esse grupo da sociedade permanece em situação de exclusão e resistência no mercado trabalhista. Diante desse cenário, é fundamental compreender as causas desse revés, dentre as quais a desigualdade social e o patriarcalismo enraizado na sociedade são fatores agravantes dessa problemática.\n\nEm uma primeira análise, destaca-se a desigualdade social como um dos fatores a ser enfrentado para amenizar a invisibilidade do trabalho de cuidado realizado pela mulher no Brasil. Nesse sentido, a escritora Carolina Maria de Jesus, em sua obra literária "Quarto de Despejo", escrita em meados do século XX, traz uma denúncia social, 